In [1]:
import os, sys, glob, logging

import pandas, scipy, numpy

import time, datetime, pprint


In [2]:
logging.basicConfig(level=logging.INFO)
# logging.info('logging info')
# logging.warning('logging.warning')


import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode

# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [3]:

logging.info("logging")


# GTEx data from https://gtexportal.org/home/datasets
# Gene transcripts per million data
cache_dir = '/home/zeno/c_drive/Users/Zeno/Downloads'

# Detailed data file:
# https://storage.googleapis.com/gtex_analysis_v7/rna_seq_data/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz
gtex_tpm_fn = os.path.join(cache_dir, 'GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz')

# Sample annotation data
# Main sample data of interest is:
# 'SMTS': 'Tissue Type, area from which the tissue sample was taken.  This is a parent value to SMTSD.'
# 'SMTSD': 'SMTS Detailed'
gtex_attr_fn = os.path.join(cache_dir, 'GTEx_v7_Annotations_SampleAttributesDS.txt')
gs = pandas.read_csv(gtex_attr_fn, sep='\t')

tissue_types = sorted(gs['SMTS'].unique())

tissue_dict = dict()
for tissue in tissue_types:
    tissue_dict[tissue] = sorted(gs[gs['SMTS'] == tissue]['SMTSD'].unique())

tissue_subtypes = sorted(gs['SMTSD'].unique())

print(tissue_types)

# Just getting a few rows of this giant file
#gtpm_head = pandas.read_table(gtex_tpm_fn, skiprows=2, sep='\t', nrows=1000, usecols=range(3))
GTPM_SKIP = 2
gtpm_ = pandas.read_table(gtex_tpm_fn, skiprows=2, sep='\t', usecols=range(4))

# Other annotation data

# Mutually exclusive 
# Which study was this from?
mut_info_fn = os.path.join(cache_dir, 'enrichments-analysis-result.txt')
#mut_info = pandas.read_csv(mut_info_fn, sep='\t')


# Genes of interest
TEST_GENES = [
    'FAM138A',  # highly expressed
    'TP53'
]

INFO:root:logging


['Adipose Tissue', 'Adrenal Gland', 'Bladder', 'Blood', 'Blood Vessel', 'Bone Marrow', 'Brain', 'Breast', 'Cervix Uteri', 'Colon', 'Esophagus', 'Fallopian Tube', 'Heart', 'Kidney', 'Liver', 'Lung', 'Muscle', 'Nerve', 'Ovary', 'Pancreas', 'Pituitary', 'Prostate', 'Salivary Gland', 'Skin', 'Small Intestine', 'Spleen', 'Stomach', 'Testis', 'Thyroid', 'Uterus', 'Vagina']


In [4]:
def gtpm_row_by_genename(gene_name):
    sub_table = gtpm_[gtpm_['Description'] == gene_name]
    return [GTPM_SKIP + 1 + i for i in sub_table.index][0]

def gtpm_skip_rows(row_ndx, gene_list=TEST_GENES):
    if row_ndx == GTPM_SKIP:
        return False  # header
    if row_ndx in range(GTPM_SKIP):
        return True
    for gn in gene_list:
        if row_ndx in gtpm_row_by_genename(gn):
            return False
    return True

def gtpm_keep_rows(gene_list=TEST_GENES):
    print(gene_list)
    return [GTPM_SKIP] + [gtpm_row_by_genename(gene) for gene in gene_list]

keep_rows = gtpm_keep_rows()
print(keep_rows)

['FAM138A', 'TP53']
[2, 6, 43806]


In [10]:
print(keep_rows)

start_time = time.time()
gtpm = pandas.read_table(gtex_tpm_fn, skiprows=lambda x: x not in keep_rows, sep='\t', header=0, usecols=range(100))

#gtpm = pandas.read_table(gtex_tpm_fn, skiprows=gtpm_skip_rows, sep='\t',)
end_time = time.time()

print("Tmp table took {} minutes".format((end_time - start_time)/60))


[2, 6, 43806]
Tmp table took 0.6716208497683207 minutes


In [11]:
gtpm

,Name,Description,GTEX-1117F-0226-SM-5GZZ7,GTEX-111CU-1826-SM-5GZYN,GTEX-111FC-0226-SM-5N9B8,GTEX-111VG-2326-SM-5N9BK,GTEX-111YS-2426-SM-5GZZQ,GTEX-1122O-2026-SM-5NQ91,GTEX-1128S-2126-SM-5H12U,GTEX-113IC-0226-SM-5HL5C,...,GTEX-13IVO-0226-SM-5LZXU,GTEX-13JUV-2826-SM-5LZWU,GTEX-13JVG-0226-SM-5J1MW,GTEX-13N11-2626-SM-5K7UQ,GTEX-13N1W-0226-SM-5K7W6,GTEX-13N2G-2726-SM-5J1MG,GTEX-13NYS-0226-SM-5MR49,GTEX-13NZ8-0926-SM-5MR3U,GTEX-13NZ9-0526-SM-5J1OX,GTEX-13NZA-0126-SM-5K7UH
0,ENSG00000237613.2,FAM138A,0.05045,0.00,0.02945,0.0326,0.00,0.0,0.0861,0.131,...,0.06036,0.0411,0.06147,0.032,0.02942,0.00,0.00,0.00,0.03722,0.06112
1,ENSG00000141510.11,TP53,24.98000,35.05,40.62000,29.1500,20.39,19.8,25.2900,35.170,...,30.19000,38.7800,24.45000,26.600,22.83000,27.28,21.28,25.23,40.54000,26.52000


In [ ]:
start_time = time.time()
time.sleep(3)
print(time.time() - start_time)

In [12]:

gt = gtpm[gtpm.columns[1:]]
gt.set_index('Description', inplace=True)
gtt = gt.transpose()
gtt.head()

Description,FAM138A,TP53
GTEX-1117F-0226-SM-5GZZ7,0.05045,24.98
GTEX-111CU-1826-SM-5GZYN,0.00000,35.05
GTEX-111FC-0226-SM-5N9B8,0.02945,40.62
GTEX-111VG-2326-SM-5N9BK,0.03260,29.15
GTEX-111YS-2426-SM-5GZZQ,0.00000,20.39


In [8]:
gtt.head().iplot(kind='hist', yTitle='TPM')

In [15]:
print(gt.head())
gt.iplot(kind='hist', yTitle='TPM')


             GTEX-1117F-0226-SM-5GZZ7  GTEX-111CU-1826-SM-5GZYN  \
Description                                                       
FAM138A                       0.05045                      0.00   
TP53                         24.98000                     35.05   

             GTEX-111FC-0226-SM-5N9B8  GTEX-111VG-2326-SM-5N9BK  \
Description                                                       
FAM138A                       0.02945                    0.0326   
TP53                         40.62000                   29.1500   

             GTEX-111YS-2426-SM-5GZZQ  GTEX-1122O-2026-SM-5NQ91  \
Description                                                       
FAM138A                          0.00                       0.0   
TP53                            20.39                      19.8   

             GTEX-1128S-2126-SM-5H12U  GTEX-113IC-0226-SM-5HL5C  \
Description                                                       
FAM138A                        0.0861                     0

In [16]:
gtt['TP53'].iplot(kind='hist', yTitle='TPM')